In [129]:
"""
#Used only for Dev

import simulator, agent, simulator_interface, behavior
from importlib import reload
reload(simulator)
reload(behavior)
reload(simulator_interface)
reload(agent)

"""

<module 'agent' from 'd:\\Github\\projet_semestriel\\simple-playgrounds-epuck\\client\\agent.py'>

# Preparing the simulator

In [1]:
from simulator_interface import open_session, close_session
agents = [
    { 
        "agent_type": "predator",
        "initial_coordinates": ((0.55,0.4),1.6),
        "radius": 15,
     },
    { 
        "agent_type": "prey",
        "initial_coordinates": ((0.5,0.6),-1.6),
        "radius": 8,
     }
]
simulator, predator, prey = open_session(nb_agents=len(agents), agents=agents)

In [2]:
simulator.status()

'running'

# Testing aggression, fear and obstacle avoidance behaviors

In [3]:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(excluded_objects=['prey', 'predator'])
    return 1 - right, 1 - left, 1

In [4]:
def aggression(robot):
    tracked_objects=["prey"]
    left, right, obj_left, obj_right = robot.prox_activations(tracked_objects, return_epucks=True)
    activation = 1
    if obj_left not in tracked_objects:
        left = 0
    if obj_right not in tracked_objects:
        right = 0
    if obj_left not in tracked_objects and obj_right not in tracked_objects:
        activation = 0
    if obj_left in tracked_objects and obj_right in tracked_objects:
        left = right = 0
    return 1 - left, 1 - right, activation

def fear(robot):
    tracked_objects=["predator"]
    left, right, obj_left, obj_right = robot.prox_activations(tracked_objects, return_epucks=True)
    activation = 1
    if obj_left not in tracked_objects:
        left = 0
    if obj_right not in tracked_objects:
        right = 0
    if obj_left not in tracked_objects and obj_right not in tracked_objects:
        activation = 0
    return 1 - right, 1 - left, activation

In [5]:
predator.detach_all_behaviors()

predator.attach_behavior(obstacle_avoidance, freq=10)
predator.attach_behavior(aggression, freq=10)

predator.start_behavior(obstacle_avoidance)
predator.start_behavior(aggression)

Behavior obstacle_avoidance started
Behavior aggression started


In [6]:
prey.detach_all_behaviors()

prey.attach_behavior(obstacle_avoidance, freq=10)
prey.attach_behavior(fear, freq=10)

prey.start_behavior(obstacle_avoidance)
prey.start_behavior(fear)

Behavior obstacle_avoidance started
Behavior fear started


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\paull\anaconda3\envs\semes\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\paull\anaconda3\envs\semes\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 36, in run
    self.run()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 36, in run
    self.loop_core()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 75, in loop_core
    self.loop_core()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 75, in loop_core
    res = self.callback(self.robot)
  File "C:\Users\paull\AppData\Local\Temp/ipykernel_8236/431346895.py", line 2, in obstacle_avoidance
    res = self.callback(self.robot)
  File "C:\Users\paull\AppData\Lo

In [164]:
predator.stop_behavior(obstacle_avoidance)
predator.stop_behavior(aggression)
predator.stop()

prey.stop_behavior(obstacle_avoidance)
prey.stop_behavior(fear)
prey.stop()

Behavior obstacle_avoidance stopped
Behavior aggression stopped
Behavior obstacle_avoidance stopped
Behavior fear stopped


# Testing sphere apparition

In [9]:
simulator.start_sphere_apparition(period=5.)

Routine sphere_apparition started


In [10]:
def seek_spheres(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    left_wheel = 1 - left
    right_wheel = 1 - right
    return left_wheel, right_wheel, 1

In [11]:
predator.detach_all_behaviors()

predator.attach_behavior(seek_spheres, freq=1)
predator.start_behavior(seek_spheres)

Behavior seek_spheres started


In [12]:
simulator.stop_sphere_apparition()

Routine sphere_apparition stopped


In [24]:
predator.stop_behavior(seek_spheres)
predator.stop()

NameError: name 'seek_spheres' is not defined

In [101]:
close_session(simulator)

Exception in thread Thread-84:
Traceback (most recent call last):
  File "C:\Users\paull\anaconda3\envs\semes\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-85:
Traceback (most recent call last):
  File "C:\Users\paull\anaconda3\envs\semes\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 36, in run
    self.run()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 36, in run
    self.loop_core()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 75, in loop_core
    self.loop_core()
  File "d:\Github\projet_semestriel\simple-playgrounds-epuck\client\behavior.py", line 75, in loop_core
    res = self.callback(self.robot)
  File "C:\Users\paull\AppData\Local\Temp/ipykernel_9796/431346895.py", line 2, in obstacle_avoidance
    res = self.callback(self.robot)
  File "C:\Users\paull\AppData\L